# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [69]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [70]:
# your code here
orders=pd.read_csv('Orders.csv')
#display(orders.head())

In [71]:
display(orders)

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397919,541904,581587,22613,2011,12,5,12,pack of 20 spaceboy napkins,12,2011-12-09 12:50:00,0.85,12680,France,10.20
397920,541905,581587,22899,2011,12,5,12,children's apron dolly girl,6,2011-12-09 12:50:00,2.10,12680,France,12.60
397921,541906,581587,23254,2011,12,5,12,childrens cutlery dolly girl,4,2011-12-09 12:50:00,4.15,12680,France,16.60
397922,541907,581587,23255,2011,12,5,12,childrens cutlery circus parade,4,2011-12-09 12:50:00,4.15,12680,France,16.60


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [73]:
# your code here
#Sub Problem 1

grouped=orders.groupby(['CustomerID','Country']).sum('amount_spent')


display(grouped)



#Initialize the column Customer_type
grouped['Customer_Type']='Not Defined'
#display(orders)

#identify the 95th percentil customers and call them VIP Customers

percentile_95 = grouped['amount_spent'].quantile(0.95)

grouped.loc[grouped['amount_spent'] > percentile_95, 'Customer_Type'] = 'VIP Customers'

#
percentile_75 = grouped['amount_spent'].quantile(0.75)
print(percentile_75)

grouped.loc[(grouped['amount_spent'] < percentile_95) & (grouped['amount_spent'] > percentile_75), 'Customer_Type'] = 'Preferred Customers'

display(grouped)


,,Unnamed: 0,InvoiceNo,year,month,day,hour,Quantity,UnitPrice,amount_spent
CustomerID,Country,,,,,,,,,
12346,United Kingdom,61619,541431,2011,1,2,10,74215,1.04,77183.60
12347,Iceland,42441700,101296926,365971,1383,441,2219,2458,481.21,4310.00
12348,Finland,2807120,16869685,62324,257,111,472,2341,178.71,1797.24
12349,Italy,35444274,42165457,146803,803,73,657,631,605.10,1757.55
12350,Norway,1365627,9231629,34187,34,51,272,197,65.30,334.40
...,...,...,...,...,...,...,...,...,...,...
18280,United Kingdom,1110495,5457120,20110,30,10,90,45,47.65,180.60
18281,United Kingdom,1560699,3895248,14077,42,49,70,54,39.36,80.82
18282,United Kingdom,4642134,6838540,24132,116,60,146,103,62.39,178.05


1661.195


,,Unnamed: 0,InvoiceNo,year,month,day,hour,Quantity,UnitPrice,amount_spent,Customer_Type
CustomerID,Country,,,,,,,,,,
12346,United Kingdom,61619,541431,2011,1,2,10,74215,1.04,77183.60,VIP Customers
12347,Iceland,42441700,101296926,365971,1383,441,2219,2458,481.21,4310.00,Preferred Customers
12348,Finland,2807120,16869685,62324,257,111,472,2341,178.71,1797.24,Preferred Customers
12349,Italy,35444274,42165457,146803,803,73,657,631,605.10,1757.55,Preferred Customers
12350,Norway,1365627,9231629,34187,34,51,272,197,65.30,334.40,Not Defined
...,...,...,...,...,...,...,...,...,...,...,...
18280,United Kingdom,1110495,5457120,20110,30,10,90,45,47.65,180.60,Not Defined
18281,United Kingdom,1560699,3895248,14077,42,49,70,54,39.36,80.82,Not Defined
18282,United Kingdom,4642134,6838540,24132,116,60,146,103,62.39,178.05,Not Defined


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [74]:
# your code here

country_vip_counts = grouped[grouped['Customer_Type'] == 'VIP Customers'].groupby('Country').size()
country_vip_counts

#Solution: United Kingdom has the most VIP Customers

Country
Australia            1
Channel Islands      1
Cyprus               1
Denmark              1
EIRE                 2
Finland              1
France               9
Germany             11
Japan                2
Netherlands          1
Norway               1
Portugal             2
Singapore            1
Spain                2
Sweden               1
Switzerland          3
United Kingdom     178
dtype: int64

## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [77]:
# your code here
country_vip_preferred_counts = grouped[(grouped['Customer_Type'] == 'VIP Customers')|(grouped['Customer_Type'] == 'Preferred Customers')].groupby('Country').size()
country_vip_preferred_counts

Country
Australia            4
Austria              1
Belgium             11
Canada               1
Channel Islands      4
Cyprus               3
Denmark              3
EIRE                 3
Finland              5
France              29
Germany             39
Greece               1
Iceland              1
Israel               2
Italy                5
Japan                4
Lebanon              1
Malta                1
Netherlands          1
Norway               7
Poland               1
Portugal             7
Singapore            1
Spain                7
Sweden               2
Switzerland          9
United Kingdom     934
dtype: int64

In [ ]:
#Solution: Still the United Kingdom